In [16]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.model_selection import KFold
from utilities import cal_score

In [17]:
stack_idx = '05'
models = '1,3-7,'

### Read CV predictions and test

In [18]:
def parse_models(exp):
    exp_split = exp.split(',')
    idx_models = []
    for e in exp_split:
        if '-' in e:
            n0, n1 = e.split('-')
            idx_models.extend(list(range(int(n0), int(n1)+1, 1)))
        else:
            idx_models.append(int(e))
    return idx_models

In [19]:
idx_models = parse_models(models)

In [20]:
files_in_output = [f for f in os.listdir('output/') if os.path.isfile('output/'+f)]
files_cv = [[f for f in files_in_output if 'model-%02d-' % idx in f and 'cv' in f][0] for idx in idx_models]
files_test_one = [[f for f in files_in_output if 'model-%02d-' % idx in f and 'test-one' in f][0] \
                  for idx in idx_models]
files_test_kf = [[f for f in files_in_output if 'model-%02d-' % idx in f and 'test-kfold' in f][0] \
                 for idx in idx_models]

In [21]:
print(files_test_kf)
print(files_test_one)

['model-01-lgb-test-kfold.csv', 'model-03-lgb-feats-selection-test-kfold.csv', 'model-04-lgb-PCA-test-kfold.csv', 'model-05-lgb-wo-per-area-test-kfold.csv', 'model-06-lgb-lr0.001-test-kfold.csv', 'model-07-keras-embedding-test-kfold.csv']
['model-01-lgb-test-one.csv', 'model-03-lgb-feats-selection-test-one.csv', 'model-04-lgb-PCA-test-one.csv', 'model-05-lgb-wo-per-area-test-one.csv', 'model-06-lgb-lr0.001-test-one.csv', 'model-07-keras-embedding-test-one.csv']


In [22]:
### !!!!!!!! HACKING THE CODE HERE !!!!!!!!!! ###
files_test_one.remove('model-07-keras-embedding-test-one.csv')
files_test_one.append('model-07-keras-embedding-test-kfold.csv')
print(files_test_one)

['model-01-lgb-test-one.csv', 'model-03-lgb-feats-selection-test-one.csv', 'model-04-lgb-PCA-test-one.csv', 'model-05-lgb-wo-per-area-test-one.csv', 'model-06-lgb-lr0.001-test-one.csv', 'model-07-keras-embedding-test-kfold.csv']


In [23]:
cv = None
test_one = None
test_kf = None

for i, f in enumerate(files_cv):
    df = pd.read_csv('output/'+f)
    if cv is None:
        cv = df[['building_id','total_price_predict']].copy()
    else:
        cv = pd.merge(cv, df[['building_id','total_price_predict']], on='building_id')
    cv = cv.rename(columns = {'total_price_predict':'pred_{}'.format(idx_models[i])})
    cv[f'log_pred_{idx_models[i]}'] = np.log1p(cv[f'pred_{idx_models[i]}'])
cv = pd.merge(cv, df[['building_id','total_price']], on='building_id')
cv['log_total_price'] = np.log1p(cv['total_price'])

for i, f in enumerate(files_test_one):
    df = pd.read_csv('output/'+f)
    if test_one is None:
        test_one = df[['building_id','total_price']].copy()
    else:
        test_one = pd.merge(test_one, df[['building_id','total_price']], on='building_id')
    test_one = test_one.rename(columns = {'total_price':'pred_{}'.format(idx_models[i])})
    test_one[f'log_pred_{idx_models[i]}'] = np.log1p(test_one[f'pred_{idx_models[i]}'])

In [24]:
test_one[['building_id','pred_5']].rename(columns={'building_id':'id','pred_5':'target'})\
    .to_csv('test_5.csv',index=False)

In [25]:
cv.head()

,building_id,pred_1,log_pred_1,pred_3,log_pred_3,pred_4,log_pred_4,pred_5,log_pred_5,pred_6,log_pred_6,pred_7,log_pred_7,total_price,log_total_price
0,jre1pJhcQj91Kdky,1.255350e+07,16.345510,1.255981e+07,16.346013,1.288381e+07,16.371482,1.282213e+07,16.366683,1.247484e+07,16.339224,12309220.0,16.325859,14215011.0,16.469809
1,jcbuA8q3KPH9SzpS,9.117771e+06,16.025736,9.675385e+06,16.085096,7.095057e+06,15.774909,9.570172e+06,16.074162,9.175013e+06,16.031994,7909433.5,15.883567,7642884.5,15.849286
2,EqWJpHmkXPyfHkB8,4.491672e+06,15.317736,4.511709e+06,15.322187,4.586149e+06,15.338551,4.687745e+06,15.360462,4.562418e+06,15.333364,4827711.0,15.389883,4807053.0,15.385595
3,ihrMWGM8Nq99Uvu6,6.918995e+06,15.749781,6.878882e+06,15.743967,6.906435e+06,15.747964,6.981481e+06,15.758772,6.937283e+06,15.752421,7259765.5,15.797858,7493026.5,15.829483
4,jRhwzoVMFLFzxAX1,2.750253e+06,14.827204,2.754684e+06,14.828814,2.725200e+06,14.818053,2.838090e+06,14.858642,2.766369e+06,14.833047,2949650.8,14.897198,2846855.8,14.861726


In [26]:
X = cv[['log_pred_{}'.format(idx) for idx in idx_models]]
y = cv['log_total_price']

reg = LassoCV(alphas=[0]+list(np.logspace(-4, 3, 7)), max_iter=100000, tol=1e-6, n_jobs=-1)
reg.fit(X, y)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: C

LassoCV(alphas=[0, 0.0001, 0.0014677992676220704, 0.021544346900318846, 0.31622776601683794, 4.641588833612782, 68.12920690579622, 1000.0],
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=100000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=1e-06,
    verbose=False)

In [27]:
print(reg.alpha_)
print(reg.mse_path_)
print(reg.coef_, reg.intercept_)

0.0001
[[1.3539862  1.37718519 1.34124855]
 [1.3539862  1.37718519 1.34124855]
 [1.3539862  1.37718519 1.34124855]
 [0.11259927 0.11703478 0.10974891]
 [0.03595454 0.03809815 0.03677873]
 [0.03556187 0.03765765 0.03653478]
 [0.03555864 0.03765107 0.03654161]
 [0.03554911 0.03764234 0.03659441]]
[0.         0.27814086 0.01312845 0.27129653 0.29085225 0.15274723] -0.09449317064488127


In [28]:
for a in [0]+list(np.logspace(-4, 3, 7)):
    reg_single = Lasso(alpha=a, max_iter=100000, tol=1e-6)
    reg_single.fit(X, y)
    print(reg_single.coef_, reg.intercept_)
    print(reg_single.score(X,y))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[-0.13904431  0.29221487  0.01357291  0.27301462  0.41369209  0.15281691] -0.09449317064488127
0.9731035108458638
[0.         0.27814086 0.01312845 0.27129653 0.29085225 0.15274723] -0.09449317064488127
0.9730993470042643
[0.         0.27917245 0.010655   0.26765744 0.29449091 0.15308509] -0.09449317064488127
0.9730980742812304
[0.         0.29269405 0.         0.21225627 0.32743589 0.15665231] -0.09449317064488127
0.9728273078530136
[0.         0.36743025 0.         0.         0.22506023 0.16951731] -0.09449317064488127
0.9167269231798255
[0. 0. 0. 0. 0. 0.] -0.09449317064488127
0.0
[0. 0. 0. 0. 0. 0.] -0.09449317064488127
0.0
[0. 0. 0. 0. 0. 0.] -0.09449317064488127
0.0


In [29]:
alphas = [0, 0.0001, 0.0002, 0.0005, 0.0008, 0.001, 0.002, 0.005, 0.008, 0.01, 0.02]
gsearch = {}

folds = KFold(n_splits=3, shuffle=True, random_state=1208)
for i_fold, (itrain, ival) in enumerate(folds.split(X)): # kfold
    print('==== Fold', i_fold+1, '====')
    
    # split train, val
    X_train = X.iloc[itrain]
    X_val = X.iloc[ival]
    y_train = y.iloc[itrain]
    y_val = y.iloc[ival]
    
    # random sample - grid search
    for a in alphas:
        if a == 0:
            reg_single = LinearRegression()
        else:
            reg_single = Lasso(alpha=a, max_iter=100000, tol=1e-6)
        reg_single.fit(X_train, y_train)
        
        y_pred = reg_single.predict(X_val)
        y_pred_final = np.expm1(y_pred)
        y_true_final = np.expm1(y_val)
        score = cal_score(y_true_final, y_pred_final)

        print('alpha, score:', a, score)
        gsearch[a] = gsearch.get(a,[]) + [score]

results = [[key, np.mean(value), value] for key, value, in gsearch.items()]
results.sort(key= lambda x: x[1], reverse=True)
for item in results:
    print(item)

==== Fold 1 ====
alpha, score: 0 5904.872604114404
alpha, score: 0.0001 5924.872584279537
alpha, score: 0.0002 5924.872586001879
alpha, score: 0.0005 5926.87259060121
alpha, score: 0.0008 5923.872594457766
alpha, score: 0.001 5923.872596653028
alpha, score: 0.002 5924.872601935233
alpha, score: 0.005 5923.872568728334
alpha, score: 0.008 5917.872449180822
alpha, score: 0.01 5914.872312032804
alpha, score: 0.02 5876.871151802474
==== Fold 2 ====
alpha, score: 0 5888.873197780169
alpha, score: 0.0001 5886.873192543575
alpha, score: 0.0002 5886.873196578011
alpha, score: 0.0005 5885.873208200674
alpha, score: 0.0008 5883.873218957404
alpha, score: 0.001 5884.873225724938
alpha, score: 0.002 5884.8732554480175
alpha, score: 0.005 5895.8732947904045
alpha, score: 0.008 5893.873248324038
alpha, score: 0.01 5878.873163871372
alpha, score: 0.02 5860.872264937752
==== Fold 3 ====
alpha, score: 0 5910.877397052235
alpha, score: 0.0001 5918.877416918593
alpha, score: 0.0002 5918.877418596997
alph

In [30]:
alpha_set = results[0][0]
print(alpha_set)
if alpha_set == 0:
    reg = LinearRegression()
else:
    reg = Lasso(alpha=alpha_set, max_iter=1000000, tol=1e-6)
reg.fit(X, y)

0.0005


Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=1e-06, warm_start=False)

In [31]:
print(reg.coef_, reg.intercept_)

[0.         0.27844247 0.01240511 0.27023231 0.29191642 0.15284603] -0.08948078176987906


### Calculate cv score

In [32]:
cv_pred_final = np.zeros(X.shape[0])
for i, col in enumerate(X):
    cv_pred_final = cv_pred_final + X[col] * reg.coef_[i]
cv_pred_final = cv_pred_final + reg.intercept_
cv_pred_final = np.expm1(cv_pred_final)
cv_true_final = np.expm1(y)

In [33]:
pd.DataFrame({'a':cv_true_final,'b':cv_pred_final}).head()

,a,b
0,14215011.0,1.264624e+07
1,7642884.5,9.216287e+06
2,4807053.0,4.622191e+06
3,7493026.5,6.998615e+06
4,2846855.8,2.801267e+06


In [34]:
cal_score(cv_true_final, cv_pred_final)

5907.874460662518

### Compute submission

In [35]:
test_pred_final = pd.DataFrame({'building_id': test_one['building_id'], 'total_price': np.zeros(test_one.shape[0])})
for i, idx in enumerate(idx_models):
    test_pred_final['total_price'] = test_pred_final['total_price'] + test_one[f'log_pred_{idx}'] * reg.coef_[i]
test_pred_final['total_price'] = test_pred_final['total_price'] + reg.intercept_
test_pred_final['total_price'] = np.expm1(test_pred_final['total_price'])
test_pred_final['total_price'] = np.clip(test_pred_final['total_price'], 0, None)

test_pred_final.to_csv('output/stack_{}_{}.csv'.format(stack_idx, models), index=False)

In [36]:
#a= pd.read_csv('output/model-03-lgb-feats-selection-cv.csv')
#b= pd.read_csv('output/model-03-lgb-feats-selection-test-one.csv')

In [37]:
#a.rename(columns={'building_id':'id','total_price_predict':'target'}).to_csv('opt-pred3.csv',index=False)
#b.rename(columns={'building_id':'id','total_price':'target'}).to_csv('opt-test3.csv',index=False)